# Apache Kafka Integration + Preprocessing / Interactive Analysis with KSQL

This notebook uses the combination of Python, Apache Kafka, KSQL for Machine Learning infrastructures. 

It includes code examples using ksql-python and other widespread components from Python’s machine learning ecosystem, like Numpy, pandas, TensorFlow and Keras. 

The use case is fraud detection for credit card payments. We use a test data set from Kaggle as foundation to train an unsupervised autoencoder to detect anomalies and potential fraud in payments. Focus of this example is not just model training, but the whole Machine Learning infrastructure including data ingestion, data preprocessing, model training, model deployment and monitoring. All of this needs to be scalable, reliable and performant.

If you want to learn more about the relation between the Apache Kafka open source ecosystem and Machine Learning, please check out these two blog posts:

- [How to Build and Deploy Scalable Machine Learning in Production with Apache Kafka](https://www.confluent.io/blog/build-deploy-scalable-machine-learning-production-apache-kafka/)
- [Using Apache Kafka to Drive Cutting-Edge Machine Learning](https://www.confluent.io/blog/using-apache-kafka-drive-cutting-edge-machine-learning)

##### This notebook is not meant to be perfect using all coding and ML best practices, but just a simple guide how to build your own notebooks where you can combine Python APIs with Kafka and KSQL

### Start Backend Services (Zookeeper, Kafka, KSQL)

The only server requirement is a local KSQL server running (with Kafka broker ZK node). If you don't have it running, just use Confluent CLI:

In [ ]:
# Shows correct startup but does not work 100% yet. Better run this command from outside Jupyter if you have any problems (e.g. from Terminal)!
! confluent start ksql-server

## Data Integration and Preprocessing with Python and KSQL

First of all, create the Kafka Topic 'creditcardfraud_source' if it does not exist already:

In [ ]:
! kafka-topics --zookeeper localhost:29092 --create --topic creditcardfraud_source --partitions 3 --replication-factor 1

Then load KSQL library and initiate connection to KSQL server:

In [6]:
client.ksql('show tables')

[{'@type': 'tables',
  'statementText': 'show tables;',
  'tables': [],
  'warnings': []}]

In [1]:
from time import sleep
from json import dumps
from kafka import KafkaProducer

In [2]:
from ksql import KSQLAPI
client = KSQLAPI('http://ksqldb-server:8088')

In [3]:
producer = KafkaProducer(bootstrap_servers=['broker:29092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [8]:
from faker import Faker
fake = Faker()

In [14]:
producer.send('transactions', value={
    'transaction_id': "RF12111",
    'from_account': fake.iban(),
    'to_account': fake.iban(),
    'amount_cents': fake.pyint(),
    'created_at': fake.date_time().strftime("%Y/%m/%d, %H:%M:%S")
})

Consume source data from Kafka Topic "creditcardfraud_source":

In [5]:
client.create_stream(table_name='TRANSACTIONS',
                     columns_type=['transaction_id string',
                                   'from_account string',
                                   'to_account string',
                                   'amount_cents integer',
                                   'created_at string'],
                     topic='transactions',
                     value_format='JSON')

True

Preprocessing: 

- Filter columns which are not needed 
- Filter messages where column 'class' is empty
- Change data format to Avro for more convenient further processing


In [ ]:
client.create_stream_as(table_name='creditcardfraud_preprocessed_avro',
                     select_columns=['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class'],
                     src_table='creditcardfraud_source',
                     conditions='Class IS NOT NULL',
                     kafka_topic='creditcardfraud_preprocessed_avro',
                     value_format='AVRO')

Take a look at the creates KSQL Streams:

In [ ]:
client.ksql('show streams')

Take a look at the metadata of the KSQL Stream:

In [ ]:
client.ksql('describe CREDITCARDFRAUD_PREPROCESSED_AVRO')

Interactive query statement:

In [ ]:
query = client.query('SELECT * FROM CREDITCARDFRAUD_PREPROCESSED_AVRO LIMIT 1')

for item in query: 
    print(item)

Produce single test data manually (if you did not connect to a real data stream which produces data continuously), e.g. from terminal:

                confluent produce creditcardfraud_source

                1,"2018-12- 18T12:00:00Z","Hans",0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,"0"
                
*BE AWARE: The KSQL Python API does a REST call. This only waits a few seconds by default and then throws a timeout exception. You need to get data into the query before the timeout (e.g. by using above command).*

In [ ]:
# TODO How to embed ' ' in Python ???
# See https://github.com/bryanyang0528/ksql-python/issues/54
# client.ksql('SET \'auto.offset.reset\'=\'earliest\'');

### Additional (optional) analysis and preprocessing examples

Some more examples for possible data wrangling and preprocessing with KSQL:

- Anonymization
- Augmentation
- Merge / Join data frames

In [ ]:
query = client.query('SELECT Id, MASK_LEFT(User, 2) FROM creditcardfraud_source LIMIT 1')

for item in query: 
    print(item)

In [ ]:
query = client.query('SELECT Id, IFNULL(Class, \'-1\') FROM creditcardfraud_source LIMIT 1')

for item in query: 
    print(item)

#### Stream-Table-Join

For the STREAM-TABLE-JOIN, you first need to create a Kafka Topic 'Users' (for the corresponding KSQL TABLE 'Users):

In [ ]:
! kafka-topics --zookeeper localhost:2181 --create --topic users --partitions 3 --replication-factor 1 

Then create the KSQL Table:

In [ ]:
client.create_table(table_name='users',
                     columns_type=['userid varchar', 'gender varchar', 'regionid varchar'],
                     topic='users',
                     key='userid',
                     value_format='AVRO')

In [ ]:
client.ksql("CREATE STREAM creditcardfraud_per_user WITH (VALUE_FORMAT='AVRO', KAFKA_TOPIC='creditcardfraud_per_user') AS SELECT Time, Amount, Class FROM creditcardfraud_source c INNER JOIN USERS u on c.user = u.userid WHERE u.USERID = 1")

# Mapping from KSQL to NumPy / pandas for Machine Learning tasks

In [ ]:
import numpy as np
import pandas as pd
import json

The query below command returns a Python generator. It can be printed e.g. by reading its values via next(query) or a for loop.

Due to a current [bug in ksql-python library](https://github.com/bryanyang0528/ksql-python/issues/57), we need to to an additional line of Python code to strip out unnecessary info and change to 2D array 

In [ ]:
query = client.query('SELECT * FROM CREDITCARDFRAUD_PREPROCESSED_AVRO LIMIT 8') # Returns a Python generator object

#items = [item for item in query][:-1]        # -1 to remove last record that is a dummy msg for "Limit Reached"          
#one_record = json.loads(''.join(items))      # Join two records as one as ksql-python is splitting it into two?          
#data = [one_record['row']['columns'][2:-1]]  # Strip out unnecessary info and change to 2D array                     
#df = pd.DataFrame(data=data)   

records = [json.loads(r) for r in ''.join(query).strip().replace('\n\n\n\n', '').split('\n')]
data = [r['row']['columns'][2:] for r in records[:-1]]
#data = r['row']['columns'][2] for r in records
df = pd.DataFrame(data=data, columns=['Time',  'V1' , 'V2' , 'V3' , 'V4' , 'V5' , 'V6' , 'V7' , 'V8' , 'V9' , 'V10' , 'V11' , 'V12' , 'V13' , 'V14' , 'V15' , 'V16' , 'V17' , 'V18' , 'V19' , 'V20' , 'V21' , 'V22' , 'V23' , 'V24' , 'V25' , 'V26' , 'V27' , 'V28' , 'Amount' , 'Class'])
df

### Generate some test data 

As discussed in the step-by-step guide, you have various options. Here we - ironically - read messages from a CSV file. This is for simple demo purposes so that you don't have to set up a real continuous Kafka stream. 

In real world or more advanced examples, you should connect to a real Kafka data stream (for instance using the Kafka data generator or Kafka Connect).

Here we just consume a few messages for demo purposes so that they get mapped into a pandas dataframe:

                cat /Users/kai.waehner/git-projects/python-jupyter-apache-kafka-ksql-tensorflow-keras/data/creditcard_extended.csv | kafka-console-producer --broker-list localhost:9092 --topic creditcardfraud_source
                
You need to do this from command line because Jupyter cannot execute this in parallel to above KSQL query.

# Preprocessing with Pandas + Model Training with TensorFlow / Keras

#### BE AWARE: You need enough messages in the pandas data frame to train the model in the below cells (if you just play around with ksql-python and just add a few Kafka events, it is not a sufficient number of rows to continue. You can simply change to df = pd.read_csv("data/creditcard.csv") as shown below in this case to get a bigger data set...


This part only includes the steps required for model training of the Autoencoder with Keras and TensorFlow. 

If you want to get a better understanding of the model, take a look at the other notebook [Python Tensorflow Keras Fraud Detection Autoencoder.ipynb](http://localhost:8888/notebooks/Python%20Tensorflow%20Keras%20Fraud%20Detection%20Autoencoder.ipynb) which includes many more details, plots and explanations.

[Kudos to David Ellison](https://www.datascience.com/blog/fraud-detection-with-tensorflow).

[The credit card fraud data set is available at Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data).

In [ ]:
# import packages
# matplotlib inline
#import pandas as pd
#import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [ ]:
# Use the dataframe from above (imported and preprocessed with KSQL)

# As alternative directly import from a CSV file ("the normal approach without Kafka and streaming data")

# "data/creditcard_small.csv" is a very small data set (just for quick demo purpose to get a model binary)
# => replace with "data/creditcard.csv" to use a real data set to train a model with good accuracy
#df = pd.read_csv("data/creditcard.csv") 


df.head(n=5) #just to check you imported the dataset properly

In [ ]:
#set random seed and percentage of test data
RANDOM_SEED = 314 #used to help randomly select the data points
TEST_PCT = 0.2 # 20% of the data

#set up graphic style in this case I am using the color scheme from xkcd.com
rcParams['figure.figsize'] = 14, 8.7 # Golden Mean
LABELS = ["Normal","Fraud"]
#col_list = ["cerulean","scarlet"]# https://xkcd.com/color/rgb/
#sns.set(style='white', font_scale=1.75, palette=sns.xkcd_palette(col_list))

In [ ]:
normal_df = [df.Class == 0] #save normal_df observations into a separate df
fraud_df = [df.Class == 1] #do the same for frauds

In [ ]:
#data = df.drop(['Time'], axis=1) #if you think the var is unimportant
df_norm = df
df_norm['Time'] = StandardScaler().fit_transform(df_norm['Time'].values.reshape(-1, 1))
df_norm['Amount'] = StandardScaler().fit_transform(df_norm['Amount'].values.reshape(-1, 1))

In [ ]:
train_x, test_x = train_test_split(df_norm, test_size=TEST_PCT, random_state=RANDOM_SEED)
train_x = train_x[train_x.Class == 0] #where normal transactions
train_x = train_x.drop(['Class'], axis=1) #drop the class column

test_y = test_x['Class'] #save the class column for the test set
test_x = test_x.drop(['Class'], axis=1) #drop the class column

train_x = train_x.values #transform to ndarray
test_x = test_x.values

### My Jupyter Notebook crashed sometimes in the next step 'model training' (probably memory issues):

In [ ]:
# Reduce number of epochs and batch_size if your Jupyter crashes (due to memory issues)
# nb_epoch = 100
# batch_size = 128
nb_epoch = 5
batch_size = 32

input_dim = train_x.shape[1] #num of columns, 30
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [ ]:
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="models/autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(train_x, train_x,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_x, test_x),
                    verbose=1,
                    callbacks=[cp, tb]).history

In [ ]:
autoencoder = load_model('models/autoencoder_fraud.h5')


In [ ]:
test_x_predictions = autoencoder.predict(test_x)
mse = np.mean(np.power(test_x - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_y})
error_df.describe()

The binary 'models/autoencoder_fraud.h5' is the trained model which can then be deployed anywhere to do prediction on new incoming events in real time. 

# Model Deployment

This demo focuses on the combination of Python and KSQL for data preprocessing and model training. If you want to understand the relation between Apache Kafka, KSQL and Python-related Machine Learning tools like TensorFlow for model deployment and monitoring, please check out my other Github projects:

Some examples of model deployment in Kafka environments:

- [Analytic models (TensorFlow, Keras, H2O and Deeplearning4j) embedded in Kafka Streams microservices](https://github.com/kaiwaehner/kafka-streams-machine-learning-examples)
- [Anomaly detection of IoT sensor data with a model embedded into a KSQL UDF](https://github.com/kaiwaehner/ksql-udf-deep-learning-mqtt-iot)
- [RPC communication between Kafka Streams application and model server (TensorFlow Serving)](https://github.com/kaiwaehner/tensorflow-serving-java-grpc-kafka-streams)

# Appendix: Pandas analysis with above Fraud Detection Data

In [ ]:
df = pd.read_csv("data/creditcard.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.describe()

In [ ]:
df['Amount']

In [ ]:
df[0:3]

In [ ]:
df.iloc[1,1]

In [ ]:
# Takes a minute or two (big CSV file)...
#df.plot()